In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [1]:
%matplotlib inline

from pathlib import Path
import itertools

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from palmerpenguins import load_penguins

from matplottoy.data import iris
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils

spath = Path("../../../draft/figures/code/")

ModuleNotFoundError: No module named 'matplottoy'

In [ ]:
penguins = load_penguins()

In [ ]:
penguins['species'].value_counts().reset_index()

In [ ]:
table = iris.DataFrame(penguins['species'].value_counts().reset_index())

In [ ]:
fts = {'position':{'name':'index', 
                   'encoder': position.Nominal({'Adelie': 0,  'Chinstrap': 1, 'Gentoo': 2})},  
        'length': {'name': 'species'},
        'facecolors': {'name':'index', 
                       'encoder': color.Categorical({'Adelie':'#FF8C00',
                                                'Gentoo':'#159090',
                                                'Chinstrap':'#A034F0'})}}

In [ ]:
table.view()['index']

In [ ]:
fig, ax = plt.subplots()

artistv = bar.Bar(table, fts, orientation='v')
ax.add_artist(artistv)
ax.set(xlim = (-0.5, 3.5), 
       ylim = (0,200),
       xticks = [.4, 1.4, 2.4],
       xticklabels = table.view(None)['index'])
fig.savefig(spath/"bar_v.png")

In [ ]:
fig, ax = plt.subplots()

artisth = bar.Bar(table, fts, orientation='horizontal')
ax.add_artist(artisth)
ax.set(ylim = (-0.5, 3.5), 
       xlim = (0,200),
       yticks = [.4, 1.4, 2.4],
       yticklabels = table.view(None)['index'])
fig.savefig(spath/"bar_h.png")

# grouped bar

In [ ]:
penguins

In [ ]:
df = penguins.groupby(['island','species']).count()['sex'].reset_index()

In [ ]:
tb = df.pivot(index=['island'], columns=['species'], values = 'sex').reset_index().fillna(0)

In [ ]:
tb['Adelie_s'] = 'Adelie'
tb['Chinstrap_s'] = 'Chinstrap'
tb['Gentoo_s'] = 'Gentoo'

In [ ]:
 df.pivot(index=['island'], columns=['species'], values = 'sex').reset_index().plot.bar(stacked=True)

In [ ]:
bt = iris.DataFrame(tb)

In [ ]:
bt.view()

In [ ]:
cmap = color.Categorical({'Adelie':'#FF8C00', 'Gentoo':'#159090','Chinstrap':'#A034F0'})
ts = {'position': {'name':'island',
                    'encoder': position.Nominal({'Biscoe':0, 'Dream':1, 'Torgersen':2})}} 
mgts = {'length': {'name': ['Adelie', 'Chinstrap', 'Gentoo']},
        'facecolors': {'name': ['Adelie_s', 'Chinstrap_s', 'Gentoo_s'], 
                       'encoder': cmap}} 
sgts = {'length':{'name': ['Adelie']},
        'facecolors' :{'name':['Adelie_s'], 'encoder':cmap}}

In [ ]:
group_transforms =  [{'length': {'name':'Adelie'},
                      'facecolors': {'name':"Adelie_s", 'encoder':cmap}},
                     {'length': {'name':'Chinstrap'},
                      'facecolors': {'name':"Chinstrap_s", 'encoder':cmap}}, 
                     {'length': {'name':'Gentoo'},
                      'facecolors': {'name':"Gentoo_s", 'encoder':cmap}}]

In [ ]:
fig, ax = plt.subplots()
artists = bar.StackedBar(bt, ts, group_transforms[:2])
ax.add_artist(artists)
_ = ax.set(ylim = (0, 140), 
       xlim = (-.5, 3.5),   
       xlabel = fts['position']['name'], 
       ylabel = fts['length']['name'], )

In [ ]:
artists.children[0]._paths

In [ ]:
%matplotlib inline
import matplotlib.path as mpath
import matplotlib.patches as mpatches

fig, (ax1, ax2) = plt.subplots(nrows=2)
for a in artistv._paths:
    path = mpath.Path(a.vertices, a.codes, closed=True)
    patch = mpatches.PathPatch(path)
    ax1.add_patch(patch)
_ = ax1.set(ylim = (0, 160), xlim = (-.5, 3.5))
for c, col in zip(artists.children, ['C0', 'C1', 'C2']):
    for a in c._paths:
        path = mpath.Path(a.vertices, a.codes, closed=True)
        patch = mpatches.PathPatch(path, color=col)
        ax2.add_patch(patch)

_ = ax2.set(ylim = (0, 200), xlim = (-.5, 3.5))

In [ ]:
artists.children[0]._paths

In [ ]:
np.array(artists.children[1].transforms['floor'])

In [ ]:
np.array(artists.children[0].transforms['floor'])

In [ ]:
bt.view()[artists.children[0].transforms['length']['name']]

# 

In [ ]:
bt.view()[artists.children[1].transforms['length']['name']]

In [ ]:
bt.view()[artists.children[2].transforms['length']['name']]

In [ ]:
list(zip([1,2,3], [2,3], [1]))

In [ ]:
list(itertools.zip_longest([1,2,3], [2,3], [1]))